# Generate Parametric Data

This notebook produces working data outputs for the [SKA Workflows](https://github.com/top-sim/skaworkflows/) utility . 

In order to demonstrate scalability, we generate estimated computing and data requirements for various combinations of telescope usage, varying both the number of channels being observed (up to the maximum 65536) and the number of antennas/stations doing the observing (to a maximum of 512 for SKA-Low, and 197 for SKA-Mid).  

In [ ]:
%cd /home/rwb/github/sdp-par-model
import sys
from sdp_par_model import reports
from sdp_par_model.parameters.definitions import *

In [ ]:
output_dir = "/home/rwb/github/skaworkflows/skaworkflows/data/sdp-par-model_output/"

# Channels multiplier to go from coarse-grained to fine-grained channels
CMULT = 128 

# These are the 'coarse-grained' channel values. 
SKA_channels = [64, 128, 256, 512]

# 32 is an arbitrary minimum; 512 hard maximum
SKA_Low_antenna = [32, 64, 128, 256, 512]

# 64 + N, where 64 is the base number of dishes
SKA_Mid_antenna = [64, 102, 140, 197]


tel_hpsos = {'SKA1_Low':[], 'SKA1_Mid':[]}
for h in HPSOs.hpso_telescopes:
    if HPSOs.hpso_telescopes[h] == 'SKA1_Low':
        tel_hpsos['SKA1_Low'].append(h)
    else:
        tel_hpsos['SKA1_Mid'].append(h)

for c in SKA_channels:
    channels = 128*c
    print(f"{channels=}")
    for l in SKA_Low_antenna:
        print(f"{l=}")
        adjusts = {'Nf_max':channels, 'Na':l}
        reports.write_csv_hpsos(f'{output_dir}/ParametricOutput_Low_antenna-{l}_channels-{channels}.csv',tel_hpsos['SKA1_Low'], parallel=0, verbose=False, adjusts=adjusts)
    for m in SKA_Mid_antenna:
        print(f"{m=}")
        adjusts = {'Nf_max':channels, 'Na':m}
        reports.write_csv_hpsos(f'{output_dir}/ParametricOutput_Mid_antenna-{m}_channels-{channels}.csv',tel_hpsos['SKA1_Mid'], parallel=0, verbose=False, adjusts=adjusts)


# Post-processing

The .csv output that is produced by the `spd-par-model` library contains a significant amount of information that we do not need. We do post-processing/cleaning using Pandas in the [SKA Workflows system sizing](https://github.com/top-sim/skaworkflows/blob/master/skaworkflows/datagen/pandas_system_sizing.py) tool. 